In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 16384 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "codellama/CodeLlama-13b-Instruct-hf",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


...


Loading checkpoint shards:  33%|████████████████████▎                                        | 1/3 [00:10<00:20, 10.41s/it]


Loading checkpoint shards:  67%|████████████████████████████████████████▋                    | 2/3 [00:21<00:10, 10.52s/it]


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████| 3/3 [00:27<00:00,  8.59s/it]


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████| 3/3 [00:27<00:00,  9.10s/it]

codellama/CodeLlama-13b-Instruct-hf does not have a padding token! Will use pad_token = <unk>.


In [5]:
lora_rank = 16

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.3 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [ ]:
from datasets import load_dataset

# Define the paths to your dataset files
data_files = {
    'train': 'vuln_data_train.csv',
    'val': 'vuln_data_val.csv',
    'test': 'vuln_data_test.csv'
}

# Load the dataset
dataset = load_dataset('csv', data_files=data_files)

In [ ]:
import random

random.seed(42)

SYSTEM_PROMPT = [
    """You are the smartest AI solidity smart contract security auditor in the world.""",
]

PROMPTS = [
    """The given Solidity function is vulnerable. Review the code and analyze its security flaws.
    
    This is the function we need to audit:
    ```solidity
    {code}
    ```
    
Vulnerability : """,
]


def make_conversational(examples):
    code = examples['vuln_code']
    desc = examples['vuln_explanation']
    assistant_prompt = f"""{desc}"""
    selected_system_prompt = random.choice(SYSTEM_PROMPT)
    selected_user_prompt = random.choice(PROMPTS) 
    conversation = [{"role": "system", "content": selected_system_prompt}, 
                    {"role": "user", "content": selected_user_prompt.format(code=code)},
                    {"role": "assistant", "content": assistant_prompt}]
    return { "conversations" : conversation, }

dataset = dataset.map(make_conversational)
dataset = dataset.remove_columns(["vuln_title", "vuln_explanation", "severity", "vuln_recommendation"])

In [8]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [9]:
dataset['train'][5]["conversations"]

[{'content': 'You are the smartest AI solidity smart contract security auditor in the world.',
  'role': 'system'},
 {'content': 'The given Solidity function is vulnerable. Review the code and analyze its security flaws.\n    \n    This is the function we need to audit:\n    ```solidity\n    function withdraw(uint256 _pid, uint256 _amount) external nonReentrant {\n        WithdrawRequest memory request = withdrawRequest[_pid][msg.sender];\n        require(request.epoch < currentEpoch() && deposits[_pid][msg.sender].epoch + 1 < currentEpoch(), "wait");\n        require(request.amount >= _amount, "too much");\n        _checkpoint(_pid, msg.sender);\n        deposits[_pid][msg.sender].amount -= uint192(_amount);\n        if (_amount > 0) {\n            IRewardStaking(convexPool[_pid]).withdrawAndUnwrap(_amount, false);\n            IERC20 lpToken = IERC20(\n                IRewardStaking(convexPool[_pid]).poolInfo(_pid).lptoken\n            );\n            lpToken.safeTransfer(msg.sender,

In [10]:
dataset['train'][5]["text"]

'<s>[INST] <<SYS>>\nYou are the smartest AI solidity smart contract security auditor in the world.\n<</SYS>>\n\nThe given Solidity function is vulnerable. Review the code and analyze its security flaws.\n    \n    This is the function we need to audit:\n    ```solidity\n    function withdraw(uint256 _pid, uint256 _amount) external nonReentrant {\n        WithdrawRequest memory request = withdrawRequest[_pid][msg.sender];\n        require(request.epoch < currentEpoch() && deposits[_pid][msg.sender].epoch + 1 < currentEpoch(), "wait");\n        require(request.amount >= _amount, "too much");\n        _checkpoint(_pid, msg.sender);\n        deposits[_pid][msg.sender].amount -= uint192(_amount);\n        if (_amount > 0) {\n            IRewardStaking(convexPool[_pid]).withdrawAndUnwrap(_amount, false);\n            IERC20 lpToken = IERC20(\n                IRewardStaking(convexPool[_pid]).poolInfo(_pid).lptoken\n            );\n            lpToken.safeTransfer(msg.sender, _amount);\n      

In [11]:
tokenized = dataset['train'].map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["conversations"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
# print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Train Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset['train'] = dataset['train'].select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Train Max Length =  1438


In [12]:
tokenized = dataset['val'].map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["conversations"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Val Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset['val'] = dataset['val'].select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Val Max Length =  1519


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

train_dataset = dataset['train']
val_dataset = dataset['val']

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False, 
    args = TrainingArguments(
        eval_strategy="steps",
        eval_steps=500,
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, 
        warmup_ratio = 0.1,
        num_train_epochs = 5, 
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", 
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        load_best_model_at_end=True,
        output_dir = "outputs_vuln_exp_codellama",
        report_to = "none", 
    ),
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<</SYS>>\n\n",
    response_part = "[/INST]",
)

In [15]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<s><s> [INST] <<SYS>>\nYou are the smartest AI solidity smart contract security auditor in the world.\n<</SYS>>\n\nThe given Solidity function is vulnerable. Review the code and analyze its security flaws.\n    \n    This is the function we need to audit:\n    ```solidity\n    function calculateSlippage(\n        uint256 unitsSold, //10^18\n        uint256 maxSlippage, //10^18\n        uint256 avgPrice, //10^18\n        LiquidationReceipt memory receipt\n    ) internal pure returns (uint256) {\n        // Check price slippage and update account states\n        if (\n            avgPrice == receipt.price || // No price change\n            (avgPrice < receipt.price && receipt.liquidationSide == Perpetuals.Side.Short) || // Price dropped, but position is short\n            (avgPrice > receipt.price && receipt.liquidationSide == Perpetuals.Side.Long) // Price jumped, but position is long\n        ) {\n            // No slippage\n            return 0;\n        } else {\n            // Liqu

In [16]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [17]:
tokenizer.decode(trainer.eval_dataset[5]["input_ids"])

"<s><s> [INST] <<SYS>>\nYou are the smartest AI solidity smart contract security auditor in the world.\n<</SYS>>\n\nThe given Solidity function is vulnerable. Review the code and analyze its security flaws.\n    \n    This is the function we need to audit:\n    ```solidity\n    function pay(\n        uint256 maturity,\n        address to,\n        address owner,\n        uint256[] memory ids,\n        uint112[] memory assetsIn,\n        uint112[] memory collateralsOut,\n        bytes calldata data\n    ) external override lock returns (uint128 assetIn, uint128 collateralOut) {\n        require(block.timestamp < maturity, 'E202');\n        require(ids.length == assetsIn.length && ids.length == collateralsOut.length, 'E205');\n        require(to != address(0), 'E201');\n        require(to != address(this), 'E204');\n\n        Pool storage pool = pools[maturity];\n\n        Due[] storage dues = pool.dues[owner];\n\n        for (uint256 i; i < ids.length; i++) {\n            Due storage du

In [18]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.eval_dataset[5]["labels"]])

"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [19]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 595 | Num Epochs = 5 | Total steps = 2,975
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 62,586,880/13,000,000,000 (0.48% trained)


Step,Training Loss,Validation Loss
500,2.125400,1.718977
1000,2.033500,1.673714
1500,1.281400,1.671195
2000,0.630700,1.706170
2500,0.586700,1.724156


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
model.save_pretrained("codellama13b_vuln_exp") 
tokenizer.save_pretrained("codellama13b_vuln_exp")

In [21]:
test_dataset = dataset['test']
test_dataset = test_dataset.map(lambda row: {'answer':row['conversations'][-1]})
test_dataset = test_dataset.map(lambda row: {'conversations':row['conversations'][:-1]})

In [22]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = test_dataset['conversations'][1]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 8192, use_cache = True,
                         temperature = 0.1, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<s> [INST] <<SYS>>\nYou are the smartest AI solidity smart contract security auditor in the world.\n<</SYS>>\n\nThe given Solidity function is vulnerable. Review the code and analyze its security flaws.\n    \n    This is the function we need to audit:\n    ```solidity\n    function deposit(uint256 _amount) external whenNotPaused {\n        _depositWithAuthorization(_amount, new bytes32[](0));\n    }\n\nfunction deposit(uint256 _amount, bytes32[] memory proof)\n        external\n        whenNotPaused\n    {\n        _depositWithAuthorization(_amount, proof);\n    }\n\nfunction _depositFor(address _recipient, uint256 _amount)\n        internal\n        nonReentrant\n    {\n        require(_recipient != address(0), "Address 0");\n        require(_amount != 0, "Amount 0");\n        require(!pausedDeposit, "pausedDeposit"); // dev: deposits are paused\n\n        uint256 _pool = balance();\n        uint256 _before = token.balanceOf(address(this));\n        token.safeTransferFrom(msg.sende

In [23]:
decoded_outputs = tokenizer.batch_decode(outputs)

In [24]:
test_dataset['answer'][1]

{'content': "An attacker can become the first depositor for a recently created StakedCitadel contract, providing a tiny amount of Citadel tokens by calling deposit(1) (raw values here, 1 is 1 wei, 1e18 is 1 Citadel as it has 18 decimals). Then the attacker can directly transfer, for example, 10^6*1e18 - 1 Citadel to StakedCitadel, effectively setting the cost of 1 of the vault token to be 10^6 * 1e18 Citadel. The attacker will still own 100% of the StakedCitadel's pool being the only depositor.\n\nAll subsequent depositors will have their Citadel token investments rounded to 10^6 * 1e18, due to the lack of precision which initial tiny deposit caused, with the remainder divided between all current depositors, i.e. the subsequent depositors lose value to the attacker.\n\nFor example, if the second depositor brings in 1.9*10^6 * 1e18 Citadel, only 1 of new vault to be issued as 1.9*10^6 * 1e18 divided by 10^6 * 1e18 will yield just 1, which means that 2.9*10^6 * 1e18 total Citadel pool wi

In [25]:
test_dataset['vuln_code'][1]

'function deposit(uint256 _amount) external whenNotPaused {\n        _depositWithAuthorization(_amount, new bytes32[](0));\n    }\n\nfunction deposit(uint256 _amount, bytes32[] memory proof)\n        external\n        whenNotPaused\n    {\n        _depositWithAuthorization(_amount, proof);\n    }\n\nfunction _depositFor(address _recipient, uint256 _amount)\n        internal\n        nonReentrant\n    {\n        require(_recipient != address(0), "Address 0");\n        require(_amount != 0, "Amount 0");\n        require(!pausedDeposit, "pausedDeposit"); // dev: deposits are paused\n\n        uint256 _pool = balance();\n        uint256 _before = token.balanceOf(address(this));\n        token.safeTransferFrom(msg.sender, address(this), _amount);\n        uint256 _after = token.balanceOf(address(this));\n        _mintSharesFor(_recipient, _after - _before, _pool);\n    }\n\nfunction _mintSharesFor(\n        address recipient,\n        uint256 _amount,\n        uint256 _pool\n    ) internal 

In [26]:
import pandas as pd

df_test = pd.read_csv("vuln_data_test_v6.csv")
df_test = df_test.groupby("vuln_code").agg({
    "vuln_title": list,  # Collect all vuln_titles in a list
    "vuln_explanation": list  # Optional: collect explanations too
}).reset_index()

def make_explanation(row):
    all_explanation = []
    for desc in row['vuln_explanation']:
        item = f"""{desc}"""
        all_explanation.append(item)
    ctx = "\n\n".join([x for x in all_explanation])
    return ctx

df_test['full_context'] = df_test.apply(make_explanation, axis=1)

In [ ]:
from datasets import load_dataset

# Define the paths to your dataset files
data_files = {
    'train': 'vuln_data_train.csv',
    'val': 'vuln_data_val.csv',
    'test': 'vuln_data_test.csv'
}

# Load the dataset
dataset = load_dataset('csv', data_files=data_files)

import random

random.seed(42)

SYSTEM_PROMPT = [
    """You are the smartest AI solidity smart contract security auditor in the world.""",
]

PROMPTS = [
    """The given Solidity function is vulnerable. Review the code and analyze its security flaws.
    
    This is the function we need to audit:
    ```solidity
    {code}
    ```
    
Vulnerability Explanation: """,
]

def make_conversational(examples):
    code = examples['vuln_code']
    title = examples['vuln_title']
    desc = examples['vuln_explanation']
    assistant_prompt = f"""Title: {title}\n\nDescription:\n{desc}"""
    selected_system_prompt = random.choice(SYSTEM_PROMPT)
    selected_user_prompt = random.choice(PROMPTS) 
    conversation = [{"role": "system", "content": selected_system_prompt}, 
                    {"role": "user", "content": selected_user_prompt.format(code=code)},
                    {"role": "assistant", "content": assistant_prompt}]
    return { "conversations" : conversation, }

dataset = dataset.map(make_conversational)
dataset = dataset.remove_columns(["vuln_title", "vuln_explanation", "severity", "vuln_recommendation"])

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)

test_dataset = dataset['test']
test_dataset = test_dataset.map(lambda row: {'answer':row['conversations'][-1]})
test_dataset = test_dataset.map(lambda row: {'conversations':row['conversations'][:-1]})

In [ ]:
import re
from unsloth.chat_templates import get_chat_template
from tqdm import tqdm  # Import tqdm for progress bar
import pandas as pd

def extract_llm_res(raw_llm):
    # Regex pattern to extract the assistant's response after the "Vulnerability Explanation" section
    pattern = r'\[/INST](.*?)</s>'

    match = re.search(pattern, raw_llm, re.DOTALL)

    if match:
        assistant_response = match.group(1).strip()
        return assistant_response
    else:
        print(f"No response found. Response: {raw_llm}")

# Initialize the result collection
results = []

# Iterate over the dataset with tqdm for progress bar
for idx, example in tqdm(enumerate(test_dataset), total=len(test_dataset), desc="Processing dataset"):
    messages = example['conversations']  # Get the conversation for the current entry
    
    # Tokenize input
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,  # Must add for generation
        return_tensors="pt",
    ).to("cuda")

    # Get LLM output
    outputs = model.generate(input_ids=inputs, max_new_tokens=2048, use_cache=True, repetition_penalty=1.2,
                                  temperature=0.1, min_p=0.1)
    raw_out = tokenizer.batch_decode(outputs)[0]
    
    # Extract LLM response
    llm_output = extract_llm_res(raw_out)
    
    # Get the ground truth for the current entry
    ground_truth = df_test[df_test['vuln_code'] == example['vuln_code']]['full_context'].values[0]
    
    results.append({
        "vuln_code": example['vuln_code'],
        "ground_truth": ground_truth,
        "llm_output": llm_output
    })

# After the loop, you can convert the results list to a DataFrame or another suitable format if needed
verification_results_df = pd.DataFrame(results)


Processing dataset:   0%|                                                                          | 0/143 [00:00<?, ?it/s]


Processing dataset:   1%|▍                                                                 | 1/143 [00:10<24:31, 10.36s/it]


Processing dataset:   1%|▉                                                               | 2/143 [00:54<1:11:20, 30.36s/it]


Processing dataset:   2%|█▍                                                                | 3/143 [01:11<56:38, 24.28s/it]


Processing dataset:   3%|█▊                                                                | 4/143 [01:21<42:33, 18.37s/it]

...


Processing dataset:  98%|██████████████████████████████████████████████████████████████▋ | 140/143 [44:26<00:32, 10.75s/it]


Processing dataset:  99%|███████████████████████████████████████████████████████████████ | 141/143 [45:00<00:35, 17.66s/it]


Processing dataset:  99%|███████████████████████████████████████████████████████████████▌| 142/143 [45:36<00:23, 23.07s/it]


Processing dataset: 100%|████████████████████████████████████████████████████████████████| 143/143 [45:49<00:00, 20.13s/it]


Processing dataset: 100%|████████████████████████████████████████████████████████████████| 143/143 [45:49<00:00, 19.23s/it]

In [36]:
verification_results_df.to_csv("sft_codellama13b_vuln_exp.csv")